In [1]:
import requests
import pandas as pd
import re
from tqdm import tqdm
import time
import os
import pysolr
import requests
import json
from deep_translator import GoogleTranslator
from langdetect import detect
from urllib.parse import urlencode
import urllib
import string

In [2]:
data = pd.read_json("./chitchat_dataset.json").to_dict(orient="records")

In [3]:
data

[{'utterance': 'Hello How are you doing today? ',
  'response': 'whats up MD im doing good how are you doing? '},
 {'utterance': 'whats up MD im doing good how are you doing? ',
  'response': 'Im alright, I just took a nap. But it was one of those naps that doesnt help anything. It just makes everything worse and you question all your life choices '},
 {'utterance': 'Im alright, I just took a nap. But it was one of those naps that doesnt help anything. It just makes everything worse and you question all your life choices ',
  'response': 'oh wow haha so you still feel tired huh? '},
 {'utterance': 'oh wow haha so you still feel tired huh? ',
  'response': 'Yeah '},
 {'utterance': 'Yeah ', 'response': 'did you go to bed late? '},
 {'utterance': 'did you go to bed late? ',
  'response': 'I have more of a head ache than anything else '},
 {'utterance': 'I have more of a head ache than anything else ',
  'response': 'dude thats terrible '},
 {'utterance': 'dude thats terrible ',
  'respons

In [4]:
GCP_IP = "34.134.102.49"


# def delete_core(core=CORE_NAME):
#     print(os.system('sudo su - solr -c "/opt/solr/bin/solr delete -c {core}"'.format(core=core)))


# def create_core(core=CORE_NAME):
#     print(os.system(
#         'sudo su - solr -c "/opt/solr/bin/solr create -c {core} -n data_driven_schema_configs"'.format(
#             core=core)))



class Indexer:
    def __init__(self, CORE_NAME):
        self.CORE_NAME = CORE_NAME
        self.solr_url = f'http://{GCP_IP}:8983/solr/'
        self.connection = pysolr.Solr(self.solr_url + CORE_NAME, always_commit=True, timeout=50000)

    # def do_initial_setup(self):
    #     delete_core()
    #     create_core()

    def create_documents(self, docs):
        print(self.connection.add(docs))

    def add_fields(self):
        data = {"add-field": [
                {
                    "name": "utterance",
                    "type": "text_en",
                    "multiValued": False
                },
                {
                    "name": "response",
                    "type": "text_en",
                    "multiValued": False
                },
            ]
        }
        # print(json.dumps(data))
        # headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        print(requests.post(self.solr_url + self.CORE_NAME + "/schema", json=data).json())

    def replace_BM25(self, b=None, k1=None):
        data = {
            "replace-field-type": [
                {
                    'name': 'text_en',
                    'class': 'solr.TextField',
                    'indexAnalyzer': {
                        'tokenizer': {
                            'class': 'solr.StandardTokenizerFactory'
                        },
                        'filters': [ {
                            'class': 'solr.LowerCaseFilterFactory'
                        }, {
                            'class': 'solr.StopFilterFactory',
                            'words': 'lang/stopwords_en.txt',
                            'ignoreCase': 'true'
                        }, {
                            'class': 'solr.EnglishPossessiveFilterFactory'
                        }, {
                            'class': 'solr.EnglishMinimalStemFilterFactory'
                        }, {
                            'class': 'solr.RemoveDuplicatesTokenFilterFactory'
                        }, {
                            'class': 'solr.PorterStemFilterFactory'
                        }]
                    },
                    'similarity': {
                        'class': 'solr.BM25SimilarityFactory',
                        'b': str(b),
                        'k1': str(k1)
                    },
                    'queryAnalyzer': {
                        'tokenizer': {
                            'class': 'solr.StandardTokenizerFactory'
                        },
                        'filters': [ {
                            'class': 'solr.LowerCaseFilterFactory'
                        }, {
                            'class': 'solr.StopFilterFactory',
                            'words': 'lang/stopwords_en.txt',
                            'ignoreCase': 'true'
                        }, {
                            'class': 'solr.EnglishMinimalStemFilterFactory'
                        }, {
                            'class': 'solr.SynonymGraphFilterFactory',
                            'expand': 'true',
                            'ignoreCase': 'true',
                            'synonyms': 'synonyms.txt'
                        }, {
                            'class': 'solr.EnglishPossessiveFilterFactory'
                        }, {
                            'class': 'solr.EnglishMinimalStemFilterFactory'
                        }, {
                            'class': 'solr.RemoveDuplicatesTokenFilterFactory'
                        }, {
                            'class': 'solr.PorterStemFilterFactory'
                        }]
                    }
                }, 
            ]
        }
        print(requests.post(self.solr_url + self.CORE_NAME + "/schema", json=data).json())

In [6]:
i_bm25 = Indexer(CORE_NAME= "chitchat")
# i_bm25.add_fields()

In [7]:
i_bm25.replace_BM25(b = 0.75, k1=1.2)

{'responseHeader': {'status': 0, 'QTime': 180}}


In [10]:
i_bm25.create_documents(data)

{
  "responseHeader":{
    "status":0,
    "QTime":19679}}

